In [50]:
# import the necessary packages
from imutils import face_utils
from datetime import datetime
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
from playsound import playsound

class Eye:
    
    side = ''
    
    left = [0, 0]
    right = [0, 0]
    top = [0, 0]
    bottom = [0, 0]

    corner_left_bot = [0, 0]
    corner_right_top = [0, 0]

    center = [0, 0]
    diff_dist = 0
    
    pupil = [0,0]
    
    left_threshold = [[0,0],[0,0]]
    right_threshold = [[0,0],[0,0]]
    

    def __init__(self, p1, p2, side):
        self.left = p2
        self.right = p1

        med_x = int((p1[0] + p2[0]) / 2)
        med_y = int((p1[1] + p2[1]) / 2)

        self.center = [med_x, med_y]

        diff_dist = self.right[0] - med_x
        self.diff_dist = diff_dist
        
        self.top = [med_x, med_y - diff_dist]
        self.bottom = [med_x, med_y + diff_dist]

        self.side = side

        self.corner_left_bot = [self.left[0] - int(diff_dist/2), med_y + diff_dist]
        self.corner_right_top = [self.right[0] + int(diff_dist/2), med_y - diff_dist]
        
        self.left_threshold = [
            [self.left[0] + int(diff_dist/2), self.top[1]],
            [self.left[0] + int(diff_dist/2), self.bottom[1]]
        ]
        
        self.right_threshold = [
            [self.right[0] - int(diff_dist/2), self.top[1]],
            [self.right[0] - int(diff_dist/2), self.bottom[1]]
        ]
       
        
    def setPupil(self,frame, x, y):
        self.pupil = [x + self.corner_left_bot[0], y + self.corner_right_top[1]]
        return self.checkPupilPosition(frame)
        
    def checkPupilPosition(self, frame):
        
        pupil_x = self.pupil[0]
        pupil_y = self.pupil[1]
        
        if ( pupil_x > self.corner_left_bot[0] and  pupil_x < self.left_threshold[0][0]):
            if( pupil_y > self.top[1] and  pupil_y < self.bottom[1]):
                return "l"
                    
        elif ( pupil_x < self.corner_right_top[0] and  pupil_x > self.right_threshold[0][0]):
            if( pupil_y > self.top[1] and  pupil_y < self.bottom[1]):
                return "r"

    def calibrateThreshold(self, left, right):
        
        if left < 10:
            left = 10
        
        if right < 10:
            right = 10
            
        left = left/10
        right = right/10
        
        self.left_threshold[0][0] = self.left[0] + int(self.diff_dist/left)
        self.left_threshold[1][0] = self.left[0] + int(self.diff_dist/left)
        
        self.right_threshold[0][0] = self.right[0] - int(self.diff_dist/right)
        self.right_threshold[1][0] = self.right[0] - int(self.diff_dist/right)

    def draw(self, frame):
        return cv2.rectangle(frame, (self.corner_left_bot[0], self.corner_left_bot[1]), (self.corner_right_top[0], self.corner_right_top[1]),
                             (255, 255, 255), cv2.FILLED)

    def identify(self, frame):
        cv2.putText(frame, self.side, (self.center[0] - 10, self.center[1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
        cv2.circle(frame, tuple(self.left), 1, (0, 0, 255), -1)
        cv2.circle(frame, tuple(self.right), 1, (0, 0, 255), -1)
        cv2.rectangle(frame, (self.corner_left_bot[0], self.corner_left_bot[1]), (self.corner_right_top[0], self.corner_right_top[1]),
                      (0, 255, 0), 1)

        cv2.circle(frame, tuple(self.corner_right_top), 1, (0, 0, 255), -1)
        cv2.circle(frame, tuple(self.corner_left_bot), 1, (0, 0, 255), -1)
        
        cv2.line(frame, tuple(self.left_threshold[0]), tuple(self.left_threshold[1]), (255, 0, 0), 1)
        cv2.line(frame, tuple(self.right_threshold[0]), tuple(self.right_threshold[1]), (255, 0, 0), 1)
        
        cv2.circle(frame, (self.pupil[0], self.pupil[1]), 1, (0, 0, 255), -1)

In [55]:
def non_linear_rootMod2(img, a, b):
    
    img_copy = img.copy().astype(np.float32)/255.0
    res_a = cv2.pow(img_copy,0.5)
    res_a = cv2.multiply(res_a, a)
    res = cv2.add(res_a,b)
    
    res[res<0] = 0
    res = res*255.0
    res[res>255] = 255
    
    res = res.astype(np.uint8)
    res = cv2.medianBlur(res,5)
    
    return res

def total(img, threshold_low_lim): 
    
    img = cv2.equalizeHist(img)
    
    a = 1.3
    b = -0.1
    params = [a,b]
    img = non_linear_rootMod2(img, a, b)
    
    rows, cols = img.shape
    
    ret, res = cv2.threshold(img,threshold_low_lim,255,cv2.THRESH_BINARY_INV)
    
    contours, _ = cv2.findContours(res, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    
    x = 0
    y = 0
    
    if contours:
        cnt = contours[0]
        (x,y), radius = cv2.minEnclosingCircle(cnt)
        x = int(x)
        y = int(y)
        radius = int(radius)
        center = (int(x), int(y))
        
        cv2.circle(res, center, radius, (255,255,255),2) 
         
        cv2.rectangle(img, (x - radius, y - radius), (x + radius, y + radius), (255, 0, 0), 1 )
        cv2.line(img, (x, 0), (x, rows), (0, 255, 0), 1)
        cv2.line(img,(0, y), (cols, y),(0,255,0),1)
        cv2.circle(img, center, 1, (255, 255, 255), -1) 
    
    return res, img, [x,y]


def nothing(*arg):
    pass

def captureAndApplyTotal(f, fparams=[], path=''):

    print("[INFO] loading facial landmark predictor...")
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

    if len(path) > 0:
        # initialize the video stream from a video file
        print("[INFO] loading video file...")
        vs = cv2.VideoCapture(path)
        time.sleep(1.0)
    else:
        # initialize the video stream and sleep for a bit, allowing the
        # camera sensor to warm up
        print("[INFO] camera sensor warming up...")
        vs = cv2.VideoCapture(0)
        time.sleep(2.0)
        
    cv2.namedWindow("Trackbars")
    cv2.createTrackbar("threshold", "Trackbars", 20, 255, nothing)
    cv2.createTrackbar("Left_threshold", "Trackbars", 20, 40, nothing)
    cv2.createTrackbar("Right_threshold", "Trackbars", 20, 40, nothing)
    
    interval = 15
    count = 0
    states = []

    while True:
        
        # grab the frame from the threaded video stream, resize it to
        # have a maximum width of 400 pixels, and convert it to
        # grayscale
        ret, frame = vs.read()
        
        if not ret:
            break
        
        frame = imutils.resize(frame, width=400)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame = cv2.flip(frame, 1)
        gray = cv2.flip(gray, 1)
        # detect faces in the grayscale frame
        rects = detector(gray, 0)
        
        
        count += 1
        if count == interval:
            left_count = states.count('l')
            right_count = states.count('r')
            
            if left_count >= 7:
                cv2.putText(frame, "LEFT", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                playsound('beep.mp3', False)
            elif right_count >= 7:
                cv2.putText(frame, "RIGHT", (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                playsound('beep.mp3', False)
                
            count = 0
            states = []
     
        # loop over the face detections
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)[0:4]

            eye_right = Eye(shape[0], shape[1], 'right')
            eye_left = Eye(shape[3], shape[2], 'left')

            oneEye = gray[eye_right.corner_right_top[1]:eye_right.corner_left_bot[1],
                     eye_right.corner_left_bot[0]:eye_right.corner_right_top[0]]
            
            threshold_low_lim = cv2.getTrackbarPos("threshold", "Trackbars")
            threshold_gaze_l = cv2.getTrackbarPos("Left_threshold", "Trackbars")
            threshold_gaze_r = cv2.getTrackbarPos("Right_threshold", "Trackbars")
            
            eye_right.calibrateThreshold(threshold_gaze_l, threshold_gaze_r)
            eye_left.calibrateThreshold(threshold_gaze_l, threshold_gaze_r)
            
            fparams = [threshold_low_lim]
            
            # Apply fn
            oneEyeTh, oneEye, pupil = f(oneEye, *fparams)

            states.append(eye_right.setPupil(frame, *pupil))
            
            oneEyeTh = cv2.resize(oneEyeTh, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
            oneEye = cv2.resize(oneEye, None, fx=5, fy=5, interpolation=cv2.INTER_LINEAR)
            
            eye_left.identify(frame)
            eye_right.identify(frame)
            
            cv2.imshow("Cropped", oneEye)
            cv2.imshow("Cropped Filter", oneEyeTh)

        # show the frame
        cv2.imshow("frame", frame)

        key = cv2.waitKey(30) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.release()

In [57]:
# path = "" => captura la webcam | path = "video/file/path/video.mp4" => captura de un archivo de video
# Demo files: demo.mp4 - muestra.mp4
path = "demo5.mp4"
# path = "demo2.mp4"
# path = "demo.mp4"
# path = "muestra.mp4"
# path = ""

captureAndApplyTotal(total, path=path)

[INFO] loading facial landmark predictor...
[INFO] loading video file...
